In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from avstack.datastructs import DataContainer
from avstack.geometry import GlobalOrigin2D
from avstack.modules.perception.detections import CentroidDetection
from avstack.modules.tracking import BasicXyTracker

In [ ]:
# set up simulation parameters
dt = 0.1
frames = 100
n_tru = 10  # number of true targets
n_fps = 6  # number of fps per frame
sx = sy = 20
sv = 2
true_states_i = np.array([[sx, sy, sv, sv]]).T * np.random.randn(
    4, n_tru
)  # states are [x, y, vx, vy]
true_states_f = (
    true_states_i
    + dt
    * frames
    * np.array([[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]])
    @ true_states_i
)

# plot the true locations of objects
fig, axs = plt.subplots(1, 2, figsize=(10, 6))
for ax, data, title in zip(axs, [true_states_i, true_states_f], ["Start", "End"]):
    ax.scatter(data[0, :], data[1, :])
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_title(f"True Object Locations {title}")
plt.show()

In [ ]:
# set up tracker
P0 = np.diag([1, 1, 5, 5]) ** 2
tracker = BasicXyTracker(
    threshold_confirmed=5,  # number of hits before track confirmed
    threshold_coast=1.0,  # amount of time without measurement before deleted
    v_max=30,  # maximum velocity allowed
    assign_metric="center_dist",  # assignment criteria
    assign_radius=1.0,  # maximum assign distance allowed
    P0=P0,  # initial track state covariance
)

# run simulation
rx = ry = 0.5**2
for i_frame in range(frames):
    t = dt * i_frame
    this_truths = true_states_i[:2, :] + t * true_states_i[2:, :]
    msmts = DataContainer(
        frame=i_frame, timestamp=t, source_identifier="sensor", data=[]
    )

    # generate detections from true objects
    m_obj = this_truths + np.array([[rx, ry]]).T * np.random.randn(2, n_tru)
    for i in range(n_tru):
        msmts.append(
            CentroidDetection(
                data=m_obj[:, i],
                noise=np.array([rx, ry]),
                source_identifier="sensor",
                reference=GlobalOrigin2D,
            )
        )

    # detections from false positives
    m_fp = np.array([[sx, sy]]).T * np.random.randn(
        2, n_fps
    )  # states are [x, y, vx, vy]
    for i in range(n_fps):
        msmts.append(
            CentroidDetection(
                data=m_fp[:, i],
                noise=np.array([rx, ry]),
                source_identifier="sensor",
                reference=GlobalOrigin2D,
            )
        )

    # pass detections to the tracker
    tracks = tracker(
        detections=msmts,
        platform=GlobalOrigin2D,
        check_reference=False,  # if you already enforce consistent reference, set to False
    )

In [ ]:
# plot tracking results
this_tracks = np.array([track.x[:2] for track in tracks]).T
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.scatter(this_truths[0, :], this_truths[1, :], marker="o", label="Truths")
ax.scatter(this_tracks[0, :], this_tracks[1, :], marker="x", label="Tracks")
ax.legend()
plt.show()